# WEB SCRAPING PRACTICE - PROXIES

## Libraries

In [105]:
import pandas as pd
from bs4 import BeautifulSoup # Librería para crear objetos tipo soup
import urllib3 # Librería para descargar el contenido de una URL definida
import numpy as np # Librería para manipular arreglos
from sqlalchemy import create_engine # Librería para crear las conexiones con la base de datos Postgre
from datetime import datetime
from datetime import timedelta, time
import pickle
import requests
import sys
import re
import random # Librería para generar números randómicos

## PROXIES DOWNLOAD

In [106]:
'''
'Procedimiento para conseguir un listado de direcciones de Free Proxy desde la WEB https://www.socks-proxy.net'
'
'********************
'
'Uso:  Se pretende conseguir un listado de Proxies gratis con sus características que permita rotar direcciones IP
'      válidas para realziar "WebScrapping" de diferentes direcciones intentando evitar el bloqueo. '
'
'Variables de salida dentro:
'
'       Se entrega un "Dataframe" denominado "datos_proxies_df" con estos campos:
'
'       "tiempo":           Char      "Contiene la fechay hora de captura formato "Sat Apr  4 13:36:22 2020"
'       "direccion_ip"      Char      "Contiene la dirección IP del Proxy en formato "XXX.XXX.XXX.XXX"
'       "puerto"            Char      "Contiene el puerto asociado a la direccion IP del proxy server"
'       "pais"              Char      "País al que parece pertenecer el Proxy"
'       "version"           Char      "Tipo de Proxy por ejemplo "Socks4"
'       "anonimo"           Char      "Confirmación de que el servidor en anónimo"
'       "https"             Char      "Contirmación de si el proxy admite navegacion (Http:) o (Http: + Https:)
'       "actualizacion"     Char.     "Indicación del tiempo que hace que se ha comprobado que funcionaba correctamente
'''

'\n\'Procedimiento para conseguir un listado de direcction de Free Proxy desde la WEB https://www.socks-proxy.net\'\n\'\n\'********************\n\'\n\'Uso:  Se pretende conseguir un listado de Proxies gratis con sus características que permita rotar direcciones IP\n\'      válidas para realziar "WebScrapping" de diferentes direcciones intentando evitar el bloqueo. \'\n\'\n\'Variables de salida dentro:\n\'\n\'       Se entrega un "Dataframe" denominado "datos_proxies_df" con estos campos:\n\'\n\'       "tiempo":           Char      "Contiene la fechay hora de captura formato "Sat Apr  4 13:36:22 2020"\n\'       "direccion_ip"      Char      "Contiene la dirección IP del Proxy en formato "XXX.XXX.XXX.XXX"\n\'       "puerto"            Char      "Contiene el puerto asociado a la direccion IP del proxy server"\n\'       "pais"              Char      "País al que parece pertenecer el Proxy"\n\'       "version"           Char      "Tipo de Proxy por ejemplo "Socks4"\n\'       "anonimo"      

In [107]:
web = "https://www.socks-proxy.net"

In [108]:
# Bloque de preparación de estructuras de datos.
columnas = ['tiempo','direccion_ip','puerto','pais', 'version','anonimo','https','actualizacion']

# Creamos un DF vacio para contener los datos de esta captura de datos
datos_proxies_df = pd.DataFrame(columns=columnas)

    
# Adquisición de la fecha actual en formato GMT
dateToProcess = datetime.utcnow()


In [109]:
# Bloque de captura de datos desde WEB.

# Imputamos una cabecera inidical para navegación inicial.
head = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# Capturamos el contenido desde la web y creamos un objeto tipo 'soup'.
r = requests.get(web, headers=head)
content = r.content
soup = BeautifulSoup(content, "html.parser")


In [110]:
# Bloque PARSING del objeto SOUP.

# Obtenemos todos los elementos de 'tbody' y nivel inferior 'tr'
tr_all = soup.select('tbody tr')


# Recorremos el objeto BS obtenido en búsqueda de las líneas y los TAGs.

for tag in tr_all:
    
    # Capturamos tiempo
    tiempo = dateToProcess
    
    # Capturamos direccion IP         
    direccion_ip = tag.find_all()[0].text
                
    # Capturamos puerto
    puerto = tag.find_all()[1].text

    # Capturamos pais
    pais = tag.find_all()[3].text
    
    # Capturamos version
    version = tag.find_all()[4].text
 
    # Capturamos anonimo
    anonimo = tag.find_all()[5].text
 
    # Capturamos https
    https = tag.find_all()[6].text
 
    # Capturamos actualizacion
    actualizacion = tag.find_all()[7].text

    #creamos un diccionario y lo añadimos al dataframe

    nueva_fila = {'tiempo':tiempo,'direccion_ip':direccion_ip, 'puerto':puerto,'pais':pais,
            'version':version,'anonimo':anonimo,'https':https,
            'actualizacion':actualizacion}
    
    #Añadimos una fila al dataset con los datos capturados
    
    datos_proxies_df = datos_proxies_df.append(nueva_fila, ignore_index=True)

proxy_list =  datos_proxies_df['direccion_ip'].to_list() # Pasamos a una lista todas las direcciones disponibles
port_list =  datos_proxies_df['puerto'].to_list() # Pasamos a una lista todos los puertos disponibles
limit = datos_proxies_df.shape[0] # Seteamos el valor máximo para la asignación randómica
proxy_index = random.randrange(limit) # Seleccionamos un dirección IP y puerto de las disponibles en la lista

## Supporting Proxies for web scraper

In [119]:
# Creación del argumento para realizar la petición mediante un proxy seleccionado
proxies = {
  'http': 'http://{}:{}'.format(proxy_list[proxy_index],port_list[proxy_index]),
  'https': 'http://{}:{}'.format(proxy_list[proxy_index],port_list[proxy_index])}

In [120]:
proxies

{'http': 'http://23.225.72.97:1080', 'https': 'http://23.225.72.97:1080'}